In [7]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import sys

model_name = '../models/cloud_classification_model_simplified3.keras'
model = tf.keras.models.load_model(model_name)

if 'simplified' in model_name:
    class_indices = {
        0: 'cirrus',
        1: 'cumulus',
        2: 'stratus'
    }
else:
    class_indices = {
        0: 'cirrus', 1: 'cirrostratus', 2: 'cirrocumulus', 
        3: 'altocumulus', 4: 'altostratus', 5: 'cumulus', 
        6: 'cumulonimbus', 7: 'nimbostratus', 8: 'stratocumulus', 
        9: 'stratus', 10: 'contrail'
    }

class_labels = {v: k for k, v in class_indices.items()}

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  
    return img_array

def predict_image(img_path):
    try:
        img_array = preprocess_image(img_path)
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions)
        predicted_class_label = class_indices[predicted_class_index].upper()
        return predicted_class_label
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")

def browse_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        predicted_label = predict_image(file_path)
        if predicted_label:
            result_label.config(text=f"Predicted Cloud Type: {predicted_label}")
            display_image(file_path)
        else:
            result_label.config(text="Prediction failed.")

def display_image(img_path):
    image = Image.open(img_path)
    image = image.resize((250, 250), resample=Image.BILINEAR) 
    photo = ImageTk.PhotoImage(image)
    image_label.config(image=photo)
    image_label.image = photo

root = tk.Tk()
root.title("Cloud Type Classifier: using  " + model_name.split('_')[-1].split('.')[0].upper())
root.geometry("600x500")

browse_button = tk.Button(root, text="Browse Image", command=browse_file)
browse_button.pack(pady=20)

result_label = tk.Label(root, text="Predicted Cloud Type: ")
result_label.pack(pady=10)

image_label = tk.Label(root)
image_label.pack(pady=20)

root.mainloop()
